# Create a project

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, kruskal
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import AutoLocator, AutoMinorLocator
import csv
import pickle
import json

## Define the parameters of your project

In [2]:
project = {
    'name': 'LysOnc',
    'data_dir': 'C:/Users/AGGVI/STAGE/', 
    'datasets': {
        'TCGA-BRCA': {
            'data': 'expression_data_tcga_brca_TCGA-BRCA_log_fpkm_1226_samples_55_genes.csv',
            'clinical': 'clinical_TCGA-BRCA.csv'
        }  
    }
}

## Define clinical groups and create the file new file for it

In [3]:
groups = {
    # --------------
    'TCGA-BRCA': {
        'NT': [{'tissue_status': ['normal']}], # Non tumour (NT) breast
        'All-tumours': [{'tissue_status': ['tumoral']}],
        'Luminal-A': [{'tissue_status': ['tumoral']}, {'pam50': ['luminal-A']}],
        'Luminal-B': [{'tissue_status': ['tumoral']}, {'pam50': ['luminal-B']}],
        'HER2-enriched': [{'tissue_status': ['tumoral']}, {'pam50': ['HER2-enriched']}],
        'Basal-like': [{'tissue_status': ['tumoral']}, {'pam50': ['basal-like']}],
        'Normal-like': [{'tissue_status': ['tumoral']}, {'pam50': ['normal-like']}],
        'Unknown': [{'tissue_status': ['tumoral']}, {'pam50': [np.nan]}],
        'T1N0': [{'ajcc_tumor_pathologic_pt_shared_stage_pathologic_categories': ['T1', 'T1a', 'T1b', 'T1c']}, {'ajcc_nodes_pathologic_pn_shared_stage_pathologic_m': ['N0', 'N0 (i-)', 'N0 (i+)']}],
        'N0': [{'ajcc_nodes_pathologic_pn_shared_stage_pathologic_m': ['N0', 'N0 (i-)', 'N0 (i+)']}],
        'M1': [{'diagnoses_1_ajcc_pathologic_m': ['M1']}],
        'Claudin-low': [{'tissue_status': ['tumoral']}, {'claudin_low': [1]}],
        },
}

## Define a function to execute queries on clinical data

In [4]:
def get_query(clinical, list_filters):
    query_and = True
    for filter_element in list_filters:
        for colname, colvalues in filter_element.items():
            query_and = query_and & (clinical[colname].isin(colvalues))
    return query_and

## Identify the samples belonging to each group and store them inside the project variable 

In [5]:
dataset_name = 'TCGA-BRCA'
project['datasets'][dataset_name]['groups'] = dict()
clinical = pd.read_csv(project['data_dir'] + project['datasets'][dataset_name]['clinical'], sep=';', index_col=0)
for group_name, list_filters in groups[dataset_name].items():
    query = get_query(clinical, list_filters)
    group_samples = list(clinical.loc[query].index)
    project['datasets'][dataset_name]['groups'][group_name] = group_samples
    print(dataset_name, group_name, len(group_samples))

TCGA-BRCA NT 113
TCGA-BRCA All-tumours 1113
TCGA-BRCA Luminal-A 547
TCGA-BRCA Luminal-B 202
TCGA-BRCA HER2-enriched 82
TCGA-BRCA Basal-like 193
TCGA-BRCA Normal-like 40
TCGA-BRCA Unknown 49
TCGA-BRCA T1N0 170
TCGA-BRCA N0 509
TCGA-BRCA M1 22
TCGA-BRCA Claudin-low 33


## Writes sample groups into a csv file

In [36]:
group_dict =  project['datasets'][dataset_name]['groups'].copy()

#inserts none values to create a csv
max_len = max(len(x) for x in group_dict.values())
for group in group_dict:
    group_dict[group].extend([None] * (max_len - len(group_dict[group])))
    
# write the dictionary to a CSV file
with open('groups.csv', 'w', newline='') as file:
    fieldnames = list(group_dict.keys()) # define fieldnames
    writer = csv.writer(file)
    for i, values in enumerate(group_dict.values()):
        row = [list(group_dict.keys())[i]] + values
        writer.writerow(row)

In [8]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, kruskal
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [9]:
test = 'ANOVA'

In [10]:
gene_list = ["A1BG", "A2M", "A2MP1", "NAT1"]

In [11]:
data = pd.read_csv('C:/Users/AGGVI/STAGE/expression_data_tcga_brca_TCGA-BRCA_log_fpkm_1226_samples_42851_genes.csv', sep = ';', index_col=0)
data.head()

,TCGA-3C-AAAU-01A,TCGA-3C-AALI-01A,TCGA-3C-AALJ-01A,TCGA-3C-AALK-01A,TCGA-4H-AAAK-01A,TCGA-5L-AAT0-01A,TCGA-5L-AAT1-01A,TCGA-5T-A9QA-01A,TCGA-A1-A0SB-01A,TCGA-A1-A0SD-01A,...,TCGA-UL-AAZ6-01A,TCGA-UU-A93S-01A,TCGA-V7-A7HQ-01A,TCGA-W8-A86G-01A,TCGA-WT-AB41-01A,TCGA-WT-AB44-01A,TCGA-XX-A899-01A,TCGA-XX-A89A-01A,TCGA-Z7-A8R5-01A,TCGA-Z7-A8R6-01A
gene_symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,0.217975,0.273098,0.200881,0.115300,0.195600,0.114900,0.089227,0.095452,0.075327,0.128557,...,0.019061,0.243425,0.073683,0.175429,0.038998,0.153935,0.119024,0.107688,0.060186,0.111432
A2M,5.582971,5.908904,6.107943,6.316021,6.243671,6.560055,6.886572,4.763502,7.957252,7.589755,...,4.912315,4.547838,5.224021,7.140991,5.607694,4.717589,7.355493,6.976987,7.805808,5.921743
A2MP1,0.000000,0.067226,0.148609,0.070939,0.052277,0.090447,0.173383,0.009347,0.198368,0.076559,...,0.027154,0.020484,0.039981,0.095857,0.238909,0.042364,0.214995,0.137241,0.293488,0.034638
NAT1,7.172849,3.535120,4.535145,4.370171,1.748805,3.909620,5.478033,4.308689,1.000216,5.034180,...,1.360926,1.217045,2.072277,6.289680,2.893556,5.735628,2.500292,4.569011,5.666995,5.308077
NAT2,0.605589,2.277300,0.050745,0.433280,0.047538,0.599984,0.179256,0.250113,0.070527,0.494160,...,0.132906,1.384989,0.513794,0.911883,0.674280,1.788017,0.191057,4.106390,0.483158,0.179129


In [12]:
expgroup = pd.read_excel('C:/Users/AGGVI/STAGE/EpiMed_experimental_grouping_2023.03.16_TCGA-BRCA.xlsx', engine='openpyxl', index_col=0)
available_samples = set(expgroup.index).intersection(set(data.columns))
expgroup = expgroup.loc[list(available_samples)]
expgroup.head()

C:\Anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,main_gse_number,id_platform,organism,sample_title,sample_source,sex,ethnic_group,age_min,age_max,id_tissue_stage,...,pr,her2,pam50,ki67_fpkm,upa_fpkm,pai1_fpkm,os_censor,dfs_censor,3-GEC,claudin_low
id_sample,,,,,,,,,,,,,,,,,,,,,
TCGA-BH-A1FM-11B,TCGA-BRCA,multi,Homo sapiens,6e6a7a6c-4630-4506-8b57-a0df12ffff4c,Solid Tissue Normal,F,not hispanic or latino,44.63,44.63,1,...,NaN,NaN,normal-like,0.692000,12.324995,10.048696,1.0,NaN,NaN,NaN
TCGA-AR-A250-01A,TCGA-BRCA,multi,Homo sapiens,0d5c5f1c-d20e-480c-a89b-8feba01caf96,Primary Tumor,F,not hispanic or latino,58.38,58.38,1,...,negative,positive,luminal-B,13.883031,95.291925,13.816098,0.0,0.0,3.0,0.0
TCGA-BH-A0W3-01A,TCGA-BRCA,multi,Homo sapiens,9ad80473-9148-45a7-ad6d-9936a899e022,Primary Tumor,F,not hispanic or latino,58.55,58.55,1,...,positive,negative,luminal-B,12.522373,9.223642,5.462926,0.0,0.0,2.0,0.0
TCGA-D8-A13Z-01A,TCGA-BRCA,multi,Homo sapiens,8caec376-1f90-4802-947f-b893c370f219,Primary Tumor,F,not hispanic or latino,51.90,51.90,1,...,negative,negative,HER2-enriched,40.146037,69.601296,26.030078,0.0,0.0,3.0,0.0
TCGA-C8-A26Y-01A,TCGA-BRCA,multi,Homo sapiens,de71b625-2b73-43a7-9bc0-89d6a13d0d52,Primary Tumor,F,not hispanic or latino,90.06,90.06,1,...,negative,negative,HER2-enriched,15.734594,30.174004,14.177491,0.0,0.0,3.0,0.0


## Read groups from csv file created earlier

In [13]:
# read in the CSV file and create a DataFrame
sampling = pd.read_csv('groups.csv', header = None)

# retrieves group names
group_names = [group for group in sampling[0]]

# create a list of Index objects, one for each row containing a group
samples = [pd.Index(data=row[2:]).dropna().set_names([row[1]]) for row in sampling.itertuples()]

In [35]:
# Performs one-way ANOVA, selecting genes from the list
with PdfPages('new.pdf') as pdf:
    for gene_name in gene_list:
        value_series = [data.loc[gene_name, sample] for sample in samples]
        if test == 'ANOVA':
            f_value, p_value = f_oneway(*value_series)
            print(f_value, p_value)
        if test == 'Kruskal-Wallis':
            f_value, p_value = kruskal(*value_series)
            print(f_value, p_value)
    
# Creates boxplot and send to pdf
        # Create a new figure
        fig, ax = plt.subplots(figsize=(10, 10))
    
        # Create a box plot
        ax.boxplot(value_series)

        # Set the font size and rotation of the tick labels on x-axis
        plt.xticks(range(1, len(group_names) + 1), group_names, rotation=45, ha='right')
            
        # Set the label of y-axis and title font sizes
        ax.set_ylabel('Expression', fontsize = 16)
        ax.set_title(f'{test} {gene_name}\n p-value = {p_value:.3e}', fontsize=18)
        
        pdf.savefig(fig)
        plt.close()

7.694183995633453 2.9459541021545347e-13
33.626540565518106 2.801482872887903e-68
4.166721240159452 3.865052518529582e-06
72.96534051226267 5.877660522739291e-146


## Save the project

In [37]:
pickle_file = f"{project['data_dir']}{project['name']}.pickle"
with open(pickle_file, 'bw') as f:
    pickle.dump(project, f)
print(pickle_file)

C:/Users/AGGVI/STAGE/LysOnc.pickle


In [38]:
json_file = pickle_file = f"{project['data_dir']}{project['name']}.json"
with open(json_file, 'w', encoding='utf-8') as f:
    json.dump(project, f, ensure_ascii=True, indent=4)
print(json_file)

C:/Users/AGGVI/STAGE/LysOnc.json
